- Suppose our boss asks us to write a function that takes a list of ints and returns some Iterable containing the elements which are prime1 numbers.

- Remember, an Iterable is just an object capable of returning its members one at a time.

- "Simple," we say, and we write the following:

In [7]:
import math

In [8]:
# function that takes an input as a number and returns whether it's prime or not

def is_prime(number):
    if number > 1:
        if number == 2:
            return True
        if number % 2 == 0:
            return False
        for current in range(3, int(math.sqrt(number) + 1), 2):
            if number % current == 0: 
                return False
        return True
    return False

In [9]:
# function that takes a list of values as an input and return the list of prime values

def get_primes(input_list):
    result_list = list()
    for elements in input_list:
        if is_prime(elements):
            result_list.append(elements)
    return result_list

input_list = [3,6,11]
print(get_primes(input_list))

[3, 11]


- Above **get_primes** implementation above fulfills the requirements, so we tell our boss we're done. 
- She reports our function works and is exactly what she wanted.

#### Dealing With Infinite Sequences

Well, not quite exactly. A few days later, our boss comes back and tells us she's run into a small problem: she wants to use our **get_primes** function on a very large list of numbers. In fact, the list is so large that merely creating it would consume all of the system's memory. To work around this, she wants to be able to call **get_primes** with a **start** value and get all the primes larger than **start** (perhaps she's solving Project Euler problem 10).

Once we think about this new requirement, it becomes clear that it requires more than a simple change to **get_primes**. Clearly, we can't return a list of all the prime numbers from **start** to infinity (operating on infinite sequences, though, has a wide range of useful applications). The chances of solving this problem using a normal function seem bleak.

Before we give up, let's determine the core obstacle preventing us from writing a function that satisfies our boss's new requirements. Thinking about it, we arrive at the following: **functions only get one chance to return results, and thus must return all results at once.** It seems pointless to make such an obvious statement; "functions just work that way," we think. The real value lies in asking, "but what if they didn't?"

Imagine what we could do if **get_primes** could simply return the next value instead of all the values at once. It wouldn't need to create a list at all. No list, no memory issues. Since our boss told us she's just iterating over the results, she wouldn't know the difference.

Unfortunately, this doesn't seem possible. Even if we had a magical function that allowed us to iterate from **n** to **infinity**, we'd get stuck after returning the first value:

```python
def get_primes(start):
    for element in magical_infinite_range(start):
        if is_prime(element):
            return element
```

Imagine get_primes is called like so:

```python
def solve_number_10():
    # She *is* working on Project Euler #10, I knew it!
    total = 2
    for next_prime in get_primes(3):
        if next_prime < 2000000:
            total += next_prime
        else:
            print(total)
            return
```

Clearly, in **get_primes**, we would immediately hit the case where **number = 3** and return at line 4. Instead of **return**, we need a way to generate a value and, when asked for the next one, pick up where we left off.

Functions, though, can't do this. When they **return**, they're done for good. Even if we could guarantee a function would be called again, we have no way of saying, "OK, now, instead of starting at the first line like we normally do, start up where we left off at line 4." Functions have a single **entry point**: the first line.

#### Enter the Generator

- This sort of problem is so common that a new construct was added to Python to solve it: the **generator**. 
- A **generator** "generates" values. Creating **generators** was made as straightforward as possible through the concept of **generator functions**, introduced simultaneously.

- A **generator function** is defined like a normal function, but whenever it needs to generate a value, it does so with the **yield** keyword rather than **return**. 
- If the body of a **def** contains **yield**, the function automatically becomes a **generator function** (even if it also contains a **return** statement). There's nothing else we need to do to create one.

**generator functions** create **generator iterators.** That's the last time you'll see the term **generator iterator**, though, since they're almost always referred to as "**generators**". Just remember that a **generator** is a special type of **iterator**. To be considered an **iterator, generators** must define a few methods, one of which is **__next__()**. To get the next value from a **generator**, we use the same built-in function as for **iterators: next()**.

This point bears repeating: **to get the next value from a generator, we use the same built-in function as for iterators: next().**

(**next()** takes care of calling the generator's **__next__()** method). Since a **generator** is a type of **iterator**, it can be used in a **for** loop.

So whenever **next()** is called on a **generator**, the **generator** is responsible for passing back a value to whomever called **next()**. It does so by calling **yield** along with the value to be passed back (e.g. **yield 7**). The easiest way to remember what yield does is to think of it as **return** (plus a little magic) for **generator functions.**

Again, this bears repeating: **yield is just return (plus a little magic) for generator functions.**

Here's a simple **generator function**:

```python
def get_primes(number):
    while True:
        if is_prime(number):
            yield number
        number += 1
```

In [11]:
import random

def get_data():
    """Return 3 random integers between 0 and 9"""
    return random.sample(range(10), 3)

def consume():
    """Displays a running average across lists of integers sent to it"""
    running_sum = 0
    data_items_seen = 0

    while True:
        data = yield
        data_items_seen += len(data)
        running_sum += sum(data)
        print('The running average is {}'.format(running_sum / float(data_items_seen)))
        
def produce(consumer):
    """Produces a set of values and forwards them to the pre-defined consumer
    function"""
    while True:
        data = get_data()
        print('Produced {}'.format(data))
        consumer.send(data)
        yield
        
if __name__ == '__main__':
    consumer = consume()
    consumer.send(None)
    producer = produce(consumer)

    for _ in range(10):
        print('Producing...')
        next(producer)

Producing...
Produced [4, 7, 1]
The running average is 4.0
Producing...
Produced [2, 6, 1]
The running average is 3.5
Producing...
Produced [7, 8, 0]
The running average is 4.0
Producing...
Produced [2, 6, 8]
The running average is 4.333333333333333
Producing...
Produced [1, 0, 8]
The running average is 4.066666666666666
Producing...
Produced [3, 6, 5]
The running average is 4.166666666666667
Producing...
Produced [0, 6, 1]
The running average is 3.9047619047619047
Producing...
Produced [6, 4, 2]
The running average is 3.9166666666666665
Producing...
Produced [9, 6, 0]
The running average is 4.037037037037037
Producing...
Produced [4, 1, 9]
The running average is 4.1


Remember...
There are a few key ideas I hope to take away from this notebook:

- Generators are used to generate a series of values
- Yield is like the return of generator functions
- The only other thing yield does is save the "state" of a generator function
- A generator is just a special type of iterator
- Like iterators, we can get the next value from a generator using next()
 - For gets values by calling next() implicitly